In [1]:
import awkward as ak
import uproot
import hist
import os
import numpy as np
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents.methods.base import NanoEventsArray
from imp import reload #每次修改调用函数后可以更新
import correctionlib, rich
import matplotlib.pyplot as plt
import sys
from coffea.lumi_tools import LumiMask

/tmp/ipykernel_306166/3220351898.py:8: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload #每次修改调用函数后可以更新


In [132]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import time

def process_root_file(file_path, Tag, ABCD,Channel):
    try:
        # 记录开始时间
        start_time = time.time()
        # 从ROOT文件创建NanoEventsArray
        events = NanoEventsFactory.from_root(file_path, treepath="Events", schemaclass=NanoAODSchema).events()
        # 提取Muon值并进行必要的处理，例如计算Muon数量
        Event = events
        Electron = events.Electron
        Photon = events.Photon
        Muon = events.Muon

        #罗列selection
        if 'ele' in Channel:
            #electron
            ele_eta = (abs(Electron.eta) < 2.5) & ~((abs(Electron.eta) > 1.4442) & (abs(Electron.eta) < 1.566 ))
            ele_pt = (Electron.pt >= 20)
            ele_cutbased_id = (Electron.cutBased >= 3)
            lg_near, lg_dr = Photon.nearest(Electron, axis=1, return_metric=True) #find the nearest electron near each photon
        elif 'muon' in Channel:
            #muon
            muon_eta = (abs(Muon.eta) < 2.4) 
            muon_pt = (Muon.pt >= 20)
            muon_id_iso = (Muon.tightId) & (Muon.pfIsoId >= 4)    
            lg_near, lg_dr = Photon.nearest(Muon, axis=1, return_metric=True) #find the nearest electron near each photon     
        else: 
            print('Wrong Channel')

        #photon
        pho_pt = (Photon.pt >= 30)
        pho_eta = (abs(Photon.eta) < 2.5) & ~((abs(Photon.eta) > 1.4442) & (abs(Photon.eta) < 1.566))
        pho_drlg = lg_dr>0.5
        pho_pixel = ~Photon.pixelSeed
        pho_genpartFlav = (Photon.genPartFlav == 1)


        #联立选择条件
        if 'ele' in Channel:
            ele_selection = ele_eta & ele_pt & ele_cutbased_id
        elif 'muon' in Channel: 
            muon_selection = muon_eta & muon_pt & muon_id_iso
        else: 
            print('Wrong Channel!')
        photon_selection = pho_pt & pho_eta & pho_drlg 
        
        if 'data' in Tag:
            photon_selection = photon_selection  
        elif 'data' not in Tag and 'dy' not in Tag:
            photon_selection = photon_selection & pho_pixel & pho_genpartFlav 
        elif 'dy' in Tag:
            photon_selection = photon_selection & (~pho_pixel | ~pho_genpartFlav)
        else:
            print('Wrong sample!')


        #处理通过object挑选的事件 要求有两个electron，两个electron异号，最大的electron pt>=25
        if 'ele' in Channel:
            good_electron = Electron[ele_selection][(ak.num(Electron[ele_selection])==2) & (ak.sum(Electron[ele_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Electron[ele_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]
            good_event = Event[(ak.num(Electron[ele_selection])==2) & (ak.sum(Electron[ele_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Electron[ele_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]
            good_photon = Photon[photon_selection][(ak.num(Electron[ele_selection])==2) & (ak.sum(Electron[ele_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Electron[ele_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]
            tem_lepton = good_electron
        elif 'muon' in Channel:
            good_muon = Muon[muon_selection][(ak.num(Muon[muon_selection])==2) & (ak.sum(Muon[muon_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Muon[muon_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]
            good_event = Event[(ak.num(Muon[muon_selection])==2) & (ak.sum(Muon[muon_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Muon[muon_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]
            good_photon = Photon[photon_selection][(ak.num(Muon[muon_selection])==2) & (ak.sum(Muon[muon_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Muon[muon_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]  
            tem_lepton = good_muon          
        else:
            print('Wrong channel!')

        #最终处理事例，调用函数，mass在质量窗内，且通过trigger
        #mass质量窗
        lepton_vectors = ak.zip(
        {
        "pt": tem_lepton["pt"],
        "eta": tem_lepton["eta"],
        "phi": tem_lepton["phi"],
        "mass": tem_lepton["mass"],
        },
        with_name="PtEtaPhiMLorentzVector",
        )
        mass_ll = (lepton_vectors[:, 0] + lepton_vectors[:, 1]).mass

        #photon id
        pt_id = (good_photon.vidNestedWPBitmap[:] & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 1) & 1) 
        scEta_id = (good_photon.vidNestedWPBitmap[:] >> 2 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 3) & 1) 
        HoE_id = (good_photon.vidNestedWPBitmap[:] >> 4 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 5) & 1) 
        siesie_id = (good_photon.vidNestedWPBitmap[:] >> 6 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 7) & 1) 
        chiso_id = (good_photon.vidNestedWPBitmap[:] >> 8 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 9) & 1) 
        neuiso_id = (good_photon.vidNestedWPBitmap[:] >> 10 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 11) & 1) 
        phoiso_id = (good_photon.vidNestedWPBitmap[:] >> 12 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 13) & 1)
        if ABCD == "A":
            all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
        elif ABCD == "B":
            all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id < 3) & (neuiso_id == 3) & (phoiso_id == 3)
        elif ABCD == "C":
            all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id < 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
        elif ABCD == "D":
            all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id < 3) & (chiso_id < 3) & (neuiso_id == 3) & (phoiso_id == 3)
        else:
            print('Wrong ABCD value!')

        if 'ele' in Channel:
            selected_good_event = good_event[(good_event.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL) & ((mass_ll >= 71) & (mass_ll <= 111)) & (ak.num(good_photon[all_id])==1)]
            selected_good_electron = good_electron[(good_event.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL) & ((mass_ll >= 71) & (mass_ll <= 111)) & (ak.num(good_photon[all_id])==1)]
            selected_good_photon = good_photon[(good_event.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL) & ((mass_ll >= 71) & (mass_ll <= 111)) & (ak.num(good_photon[all_id])==1)]
            VARIABLES = {
                'photon_pt': ak.Array(selected_good_photon[:,0].pt),
                'photon_eta': ak.Array(selected_good_photon[:,0].eta),
                'ele1_pt':ak.Array(selected_good_electron[:,0].pt),
                'run':ak.Array(selected_good_event.run),
                # 添加更多的变量...
            }
        elif 'muon' in Channel:
            selected_good_event = good_event[(good_event.HLT.Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8) & ((mass_ll >= 71) & (mass_ll <= 111)) & (ak.num(good_photon[all_id])==1)]
            selected_good_muon = good_muon[(good_event.HLT.Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8) & ((mass_ll >= 71) & (mass_ll <= 111)) & (ak.num(good_photon[all_id])==1)]
            selected_good_photon = good_photon[(good_event.HLT.Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8) & ((mass_ll >= 71) & (mass_ll <= 111)) & (ak.num(good_photon[all_id])==1)]
            VARIABLES = {
                'photon_pt': ak.Array(selected_good_photon[:,0].pt),
                'photon_eta': ak.Array(selected_good_photon[:,0].eta),
                'muon1_pt':ak.Array(selected_good_muon[:,0].pt),
                'run':ak.Array(selected_good_event.run),
                # 添加更多的变量...
            }
        else:
            print('Wrong channel!')

        # 返回结果和用时
        if 'fidu' not in Channel:
            # 记录结束时间
            end_time = time.time()
            return VARIABLES, end_time - start_time
########------------------------Fiducial part--------------------------##############
        elif 'fidu' in Channel:
            #选择条件(注意只选择zg样本)
            if 'zg' not in key:
                print('Wrong fiducial sample, ZG only!')
            Event = selected_good_event
            Lepton = selected_good_event.GenDressedLepton
            Photon = selected_good_event.GenIsolatedPhoton
            if 'ele' in Channel:
                #electron
                lepton_eta = (abs(Lepton.eta) < 2.5) & ~((abs(Lepton.eta) > 1.4442) & (abs(Lepton.eta) < 1.566 ))
            elif 'muon' in Channel:
                #muon
                lepton_eta = (abs(Lepton.eta) < 2.4)
            else:
                print('Wrong fiducial channel!')
            lepton_pt = (Lepton.pt >= 20)
            lg_near, lg_dr = Photon.nearest(Lepton, axis=1, return_metric=True) #find the nearest electron near each photon       
            pho_pt = (Photon.pt >= 30)
            pho_eta = (abs(Photon.eta) < 2.5) & ~((abs(Photon.eta) > 1.4442) & (abs(Photon.eta) < 1.566))
            pho_drlg = lg_dr>0.5

            #联立选择条件
            lepton_selection = lepton_eta & lepton_pt
            photon_selection = pho_pt & pho_eta & pho_drlg

            #处理通过object挑选的事件 要求有两个electron，两个electron异号，最大的electron pt>=25
            good_lepton = Lepton[lepton_selection][(ak.num(Lepton[lepton_selection])==2) & (ak.sum(Lepton[lepton_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Lepton[lepton_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]
            good_event = Event[(ak.num(Lepton[lepton_selection])==2) & (ak.sum(Lepton[lepton_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Lepton[lepton_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]
            good_photon = Photon[photon_selection][(ak.num(Lepton[lepton_selection])==2) & (ak.sum(Lepton[lepton_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Lepton[lepton_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])==1)]

            #进一步加函数
            tem_lepton = good_lepton
            lepton_vectors = ak.zip(
            {
            "pt": tem_lepton["pt"],
            "eta": tem_lepton["eta"],
            "phi": tem_lepton["phi"],
            "mass": tem_lepton["mass"],
            },
            with_name="PtEtaPhiMLorentzVector",
            )
            mass_ll = (lepton_vectors[:, 0] + lepton_vectors[:, 1]).mass

            selected_good_event = good_event[((mass_ll >= 71) & (mass_ll <= 111))]
            selected_good_lepton = good_lepton[((mass_ll >= 71) & (mass_ll <= 111))]
            selected_good_photon = good_photon[((mass_ll >= 71) & (mass_ll <= 111))]

            ###out fiducial part
            fiducial_events = selected_good_event.event
            outfiducial_events = Event.event

            event_mask = ~ak.any(outfiducial_events[:, None] == fiducial_events, axis=-1)

            selected_outfiducial_event = Event[event_mask]
            selected_outfiducial_photon = Photon[event_mask]
            selected_outfiducial_lepton = Lepton[event_mask]

            if 'ele' in Channel:
                VARIABLES2 = {
                    'gen_in_photon_pt': ak.Array(selected_good_photon.pt),
                    'gen_in_photon_eta': ak.Array(selected_good_photon.eta),
                    'gen_in_ele_pt': ak.Array(selected_good_lepton.pt),
                    'gen_in_ele_eta': ak.Array(selected_good_lepton.eta),
                    'gen_out_photon_pt':ak.Array(selected_outfiducial_photon.pt),
                    'gen_out_photon_eta':ak.Array(selected_outfiducial_photon.eta),
                    'gen_out_ele_pt':ak.Array(selected_outfiducial_lepton.pt),
                    'gen_out_ele_eta': ak.Array(selected_outfiducial_lepton.eta),
                    'reco_in_photon_pt': ak.Array(selected_good_event.Photon.pt),
                    'reco_in_photon_eta': ak.Array(selected_good_event.Photon.eta),
                    'reco_in_ele_pt': ak.Array(selected_good_event.Electron.pt),
                    'reco_in_ele_eta': ak.Array(selected_good_event.Electron.eta),
                    'reco_out_photon_pt':ak.Array(selected_outfiducial_event.Photon.pt),
                    'reco_out_photon_eta':ak.Array(selected_outfiducial_event.Photon.eta),
                    'reco_out_ele_pt':ak.Array(selected_outfiducial_event.Electron.pt),
                    'reco_out_ele_eta': ak.Array(selected_outfiducial_event.Electron.eta),                    

                }
                VARIABLES.update(VARIABLES2)
            elif 'muon' in Channel:
                VARIABLES = {
                    'gen_photon_pt': ak.Array(selected_good_photon.pt),
                    'gen_photon_eta': ak.Array(selected_good_photon.eta),
                    'gen_in_muon_pt': ak.Array(selected_good_lepton.pt),
                    'gen_in_muon_eta': ak.Array(selected_good_lepton.eta),
                    'gen_out_photon_pt':ak.Array(selected_outfiducial_photon.pt),
                    'gen_out_photon_eta':ak.Array(selected_outfiducial_photon.pt),
                    'gen_out_muon_pt':ak.Array(selected_outfiducial_lepton.pt),
                    'gen_out_muon_eta': ak.Array(selected_outfiducial_lepton.eta),
                    'reco_in_photon_pt': ak.Array(selected_good_event.Photon.pt),
                    'reco_in_photon_eta': ak.Array(selected_good_event.Photon.eta),
                    'reco_in_muon_pt': ak.Array(selected_good_event.Muon.pt),
                    'reco_in_muon_eta': ak.Array(selected_good_event.Muon.eta),
                    'reco_out_photon_pt':ak.Array(selected_outfiducial_event.Photon.pt),
                    'reco_out_photon_eta':ak.Array(selected_outfiducial_event.Photon.eta),
                    'reco_out_muon_pt':ak.Array(selected_outfiducial_event.Muon.pt),
                    'reco_out_muon_eta': ak.Array(selected_outfiducial_event.Muon.eta),
                }
                VARIABLES.update(VARIABLES2)
            else:
                print('Wrong Fiducial channel!')
            # 记录结束时间
            end_time = time.time()
            return VARIABLES, end_time - start_time
        else:
            print('Do you want to calc fidu xsec or not?')
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None, 0


In [137]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import input_file 
reload(input_file)

# 假设你有一个字典，key是样本名，value是对应的ROOT文件路径列表
samples_dict = input_file.input_file('CD_ZG')# choice: test, CD_ZG 

###---------------修改参数区-----------------
ABCD = 'A'
Channel = 'fidu_ele'#muon fidu_ele fidu_muon
# Channel = 'ele'#muon fidu_ele fidu_muon
max_workers = 50  # 最大并行核心数

# 所有文件的处理结果
all_results = {}

# 初始化进度条
progress = tqdm(total=sum(len(files) for files in samples_dict.values()), desc='Processing files', unit='file')

with ProcessPoolExecutor(max_workers=max_workers) as executor:
    futures = {}
    for key, files in samples_dict.items():
        for file in files:
            # 向函数传递额外的参数Tag和ABCD
            future = executor.submit(process_root_file, file, key, ABCD, Channel)
            futures[future] = key
    
    for future in as_completed(futures):
        key = futures[future]
        try:
            result_dict, duration = future.result()  # result_dict 是我们从函数中返回的字典
            if result_dict is not None:
                if key not in all_results:
                    all_results[key] = []
                all_results[key].append(result_dict)  # 存储整个字典
                # 更新进度条
                progress.update(1)
                progress.set_postfix({'last_duration': f'{duration:.2f}s', 'active_workers': f'{executor._max_workers}'})  # 显示上一个文件处理用时和活动核心数
        except Exception as e:
            print(f"Error processing future for key {key}: {e}")
            progress.update(1)

progress.close()  # 关闭进度条

# 此时 all_results 中存储的是每个文件对应的字典
print(all_results)

Processing files:   0%|          | 0/167 [00:00<?, ?file/s]

Processing files:  24%|██▍       | 40/167 [01:06<02:50,  1.34s/file, last_duration=62.93s, active_workers=50]

Error processing file /data/bond/muty/v12_2022/ZG/DYGto2LG-1Jets_MLL-50_PTG-10to50_TuneCP5_13p6TeV_amcatnloFXFX-pythia8_2022/231130_103141/0000/tree_6.root: cannot broadcast RegularArray of size 0 with RegularArray of size 1

(https://github.com/scikit-hep/awkward-1.0/blob/1.8.0/src/awkward/_util.py#L914)


Processing files:  29%|██▊       | 48/167 [01:25<04:22,  2.20s/file, last_duration=42.73s, active_workers=50]

Error processing file /data/bond/muty/v12_2022/ZG/DYGto2LG-1Jets_MLL-50_PTG-10to50_TuneCP5_13p6TeV_amcatnloFXFX-pythia8_2022/231130_103141/0000/tree_80.root: cannot broadcast RegularArray of size 0 with RegularArray of size 1

(https://github.com/scikit-hep/awkward-1.0/blob/1.8.0/src/awkward/_util.py#L914)


Processing files:  99%|█████████▉| 165/167 [05:19<00:03,  1.94s/file, last_duration=252.12s, active_workers=50]


{'zg0_22': [{'photon_pt': <Array [45.8, 34.4, 42.2, 36.5] type='4 * ?float32[parameters={"__doc__": "pt"}]'>, 'photon_eta': <Array [0.722, -0.734, 1.23, 0.55] type='4 * ?float32[parameters={"__doc__": "et...'>, 'ele1_pt': <Array [25.6, 49.1, 70.5, 70.9] type='4 * float32[parameters={"__doc__": "pt"}]'>, 'run': <Array [1, 1, 1, 1] type='4 * uint32[parameters={"__doc__": "run/i"}]'>, 'gen_in_photon_pt': <Array [[46.5], [34.2], [43.1], [37]] type='4 * var * ?float32[parameters={"__do...'>, 'gen_in_photon_eta': <Array [[0.722], [-0.736], [1.23], [0.551]] type='4 * var * ?float32[parameters=...'>, 'gen_in_ele_pt': <Array [[25.9, 25], [51.5, ... [70.6, 44.5]] type='4 * var * float32[parameters=...'>, 'gen_in_ele_eta': <Array [[-2.31, 0.3], ... [-0.301, -0.382]] type='4 * var * float32[parameters={...'>, 'gen_out_photon_pt': <Array [] type='0 * var * float32[parameters={"__doc__": "pt"}]'>, 'gen_out_photon_eta': <Array [] type='0 * var * float32[parameters={"__doc__": "eta"}]'>, 'gen_out_ele_

In [138]:
def merge_sample_data(sample_name, sample_data):
    try:
        # 初始化一个字典，用于存储当前样本的合并数据
        merged_data = {}

        # 获取第一个条目中的信息键
        info_keys = list(sample_data[0].keys())

        # 遍历每个信息键
        for info_key in info_keys:
            # 从 sample_data 中提取当前信息键的数据
            data_to_concatenate = [entry[info_key] for entry in sample_data]

            # 使用 ak.concatenate 将数据连接在一起
            merged_data[info_key] = ak.concatenate(data_to_concatenate)
        
        return merged_data
    except Exception as e:
        print(f"Error merging data for sample {sample_name}: {e}")
        return {}


In [139]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

# sample_names = ["dataC_ele","dataD_ele","zg0_22","zg1_22", "zg2_22", "zg3_22","tt_22","zz_22", "ww_22", "wz_22","dy_22"]#
# sample_names = ["zg0_22","zg1_22", "zg2_22", "zg3_22","tt_22","zz_22", "ww_22", "wz_22","dy_22"]#,
# sample_names = ["zg2_22","zg3_22","dy_22"]#,"dy""zg2", "zg3"
sample_names = ["zg0_22","zg1_22","zg2_22","zg3_22"]#,"dy""zg2", "zg3"


# 初始化进度条
progress = tqdm(total=len(sample_names), desc='Merging samples', unit='sample')

# 所有样本的合并结果
merged_results = {}

with ProcessPoolExecutor(max_workers=max_workers) as executor:
    # 提交每个样本的合并任务到进程池
    futures = {executor.submit(merge_sample_data, sample_name, all_results[sample_name]): sample_name for sample_name in sample_names}
    
    for future in as_completed(futures):
        sample_name = futures[future]
        try:
            result = future.result()
            merged_results[sample_name] = result
            # 更新进度条
            progress.update(1)
        except Exception as e:
            print(f"Error processing future for sample {sample_name}: {e}")
            progress.update(1)

progress.close()  # 关闭进度条

# 此时 merged_results 中存储的是每个样本合并后的结果
print(merged_results)


Merging samples:   0%|          | 0/4 [00:00<?, ?sample/s]

Merging samples: 100%|██████████| 4/4 [00:01<00:00,  2.55sample/s]

{'zg0_22': {'photon_pt': <Array [45.8, 34.4, 42.2, ... 35.1, 31.8, 41.6] type='24223 * ?float32[parameter...'>, 'photon_eta': <Array [0.722, -0.734, 1.23, ... -1.14, 0.876] type='24223 * ?float32[parameters...'>, 'ele1_pt': <Array [25.6, 49.1, 70.5, ... 94.2, 42.7, 59.2] type='24223 * float32[parameters...'>, 'run': <Array [1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1] type='24223 * uint32[parameters=...'>, 'gen_in_photon_pt': <Array [[46.5], [34.2], ... [31.3], [39.8]] type='23082 * var * ?float32[paramet...'>, 'gen_in_photon_eta': <Array [[0.722], [-0.736], ... -1.14], [0.874]] type='23082 * var * ?float32[par...'>, 'gen_in_ele_pt': <Array [[25.9, 25], [51.5, ... [59, 34.8]] type='23082 * var * float32[parameter...'>, 'gen_in_ele_eta': <Array [[-2.31, 0.3], ... [1.8, 0.97]] type='23082 * var * float32[parameters={"...'>, 'gen_out_photon_pt': <Array [[34.3], [43.7], ... [], [29.6]] type='1141 * var * float32[parameters={"...'>, 'gen_out_photon_eta': <Array [[2.02], [0.104], ... [], [-0.567]

In [140]:
merged_results

{'zg0_22': {'photon_pt': <Array [45.8, 34.4, 42.2, ... 35.1, 31.8, 41.6] type='24223 * ?float32[parameter...'>,
  'photon_eta': <Array [0.722, -0.734, 1.23, ... -1.14, 0.876] type='24223 * ?float32[parameters...'>,
  'ele1_pt': <Array [25.6, 49.1, 70.5, ... 94.2, 42.7, 59.2] type='24223 * float32[parameters...'>,
  'run': <Array [1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1] type='24223 * uint32[parameters=...'>,
  'gen_in_photon_pt': <Array [[46.5], [34.2], ... [31.3], [39.8]] type='23082 * var * ?float32[paramet...'>,
  'gen_in_photon_eta': <Array [[0.722], [-0.736], ... -1.14], [0.874]] type='23082 * var * ?float32[par...'>,
  'gen_in_ele_pt': <Array [[25.9, 25], [51.5, ... [59, 34.8]] type='23082 * var * float32[parameter...'>,
  'gen_in_ele_eta': <Array [[-2.31, 0.3], ... [1.8, 0.97]] type='23082 * var * float32[parameters={"...'>,
  'gen_out_photon_pt': <Array [[34.3], [43.7], ... [], [29.6]] type='1141 * var * float32[parameters={"...'>,
  'gen_out_photon_eta': <Array [[2.02], [0.104]

In [149]:
def save_event_dict(event_dict, file_prefix):
    for key, value in event_dict.items():
        for subkey, subvalue in value.items():
            ak.to_parquet(subvalue, f'{file_prefix}_{key}_{subkey}.parquet')


In [148]:
#储存
save_event_dict(merged_results, 'selection_file/CD/Electron/A/merged_results')


In [181]:
import os
import awkward as ak

def load_event_dict(directory, file_prefix, sample_names):
    # 初始化一个空的字典来存储加载的数据
    event_dict = {}

    # 遍历sample_names，为每个样本名构造文件名并加载数据
    for sample in sample_names:
        sample_dict = {}  # 存储每个样本的数据
        sample_prefix = f"{file_prefix}_{sample}"  # 构造每个样本的文件前缀

        # 列出所有与sample_prefix匹配的.parquet文件
        for filename in os.listdir(directory):
            if filename.startswith(sample_prefix) and filename.endswith('.parquet'):
                # 从文件名中提取subkey
                subkey = filename[len(sample_prefix)+1:].replace('.parquet', '')
                
                # 完整的文件路径
                full_path = os.path.join(directory, filename)
                
                # 加载parquet文件
                try:
                    sample_dict[subkey] = ak.from_parquet(full_path)
                except Exception as e:
                    print(f"读取文件时出错: {full_path}")
                    print(f"错误信息: {e}")
        
        # 如果sample_dict不为空，添加到event_dict中
        if sample_dict:
            event_dict[sample] = sample_dict
        else:
            print(f"未找到样本 {sample} 的任何数据文件。")

    return event_dict


In [182]:

# 定义样本名称
sample_names = ["zg0_22", "zg1_22", "zg2_22", "zg3_22"]

# 读取
loaded_results = load_event_dict('selection_file/CD/Electron/A', 'merged_results', sample_names)



In [183]:
loaded_results

{'zg0_22': {'photon_pt': <Array [45.8, 34.4, 42.2, ... 35.1, 31.8, 41.6] type='24223 * ?float32'>,
  'photon_eta': <Array [0.722, -0.734, 1.23, ... -1.14, 0.876] type='24223 * ?float32'>,
  'ele1_pt': <Array [25.6, 49.1, 70.5, ... 94.2, 42.7, 59.2] type='24223 * float32'>,
  'run': <Array [1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1] type='24223 * uint32'>,
  'gen_in_photon_pt': <Array [[46.5], [34.2], ... [31.3], [39.8]] type='23082 * var * ?float32'>,
  'gen_in_photon_eta': <Array [[0.722], [-0.736], ... -1.14], [0.874]] type='23082 * var * ?float32'>,
  'gen_in_ele_pt': <Array [[25.9, 25], [51.5, ... [59, 34.8]] type='23082 * var * float32'>,
  'gen_in_ele_eta': <Array [[-2.31, 0.3], ... [1.8, 0.97]] type='23082 * var * float32'>,
  'gen_out_photon_pt': <Array [[34.3], [43.7], ... [], [29.6]] type='1141 * var * float32'>,
  'gen_out_photon_eta': <Array [[2.02], [0.104], ... [], [-0.567]] type='1141 * var * float32'>,
  'gen_out_ele_pt': <Array [[58.7, 19.4], ... [58.8, 29.2]] type='1141 